## Import all necessary libraries

In [1]:
from sklearn import datasets
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

## Load Data

In [2]:
iris = datasets.load_iris()
x=iris['data']
y=iris['target']

In [3]:
y = keras.utils.to_categorical(y) #convert label to one hot encoding

## Experiment - Custom Dropout

### Define a Custom Dropout Layer

In [17]:
class CustomDropoutLayer(keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropoutLayer, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=True):
        if not training:
            return inputs  # No dropout during inference

        # Find the indices of the n lowest values in each row
        neg_tensor = -inputs  # Negate the tensor to find the smallest values using top_k
        
        shape = tf.shape(inputs)
        col = shape[1]
        tf.print("col", col)
        tf.print("inputs", inputs)

        tf.print("col type",type(col))

        num_to_drop_per_row = tf.cast(tf.math.floor(self.rate * tf.cast(col, tf.float32)), tf.int32)
        tf.print("num_to_drop_per_row",num_to_drop_per_row)

        _, indices_to_mask = tf.math.top_k(neg_tensor, k=num_to_drop_per_row)
        tf.print("indices_to_mask",indices_to_mask)

        # Create a mask for these indices
        row_indices = tf.range(tf.shape(inputs)[0])[:, tf.newaxis]  # Row indices (expand to 2D)
        row_indices = tf.tile(row_indices, [1, num_to_drop_per_row])  # Repeat row indices to match the number of elements
        full_indices = tf.stack([row_indices, indices_to_mask], axis=-1)  # Combine row and column indices
        
        # # Create a binary mask
        mask = tf.ones_like(inputs)
        updates = tf.zeros(tf.shape(indices_to_mask), dtype=inputs.dtype)
        mask = tf.tensor_scatter_nd_update(mask, full_indices, updates)  # Set n lowest values to 0

        # # Apply the mask to the tensor
        masked_tensor = inputs * mask

        tf.print("masked tensor", masked_tensor)
        return masked_tensor
        # return inputs
    
    def call_deactivate(self, inputs, training=True):
        if not training:
            return inputs  # No dropout during inference

        # Find the indices of the n lowest values in each row
        neg_tensor = -inputs  # Negate the tensor to find the smallest values using top_k
        
        shape = tf.shape(inputs)
        col = shape[1]

        num_to_drop_per_row = tf.cast(tf.math.floor(self.rate * tf.cast(col, tf.float32)), tf.int32)

        _, indices_to_mask = tf.math.top_k(neg_tensor, k=num_to_drop_per_row)

        # Create a mask for these indices
        row_indices = tf.range(tf.shape(inputs)[0])[:, tf.newaxis]  # Row indices (expand to 2D)
        row_indices = tf.tile(row_indices, [1, num_to_drop_per_row])  # Repeat row indices to match the number of elements
        full_indices = tf.stack([row_indices, indices_to_mask], axis=-1)  # Combine row and column indices
        
        # # Create a binary mask
        mask = tf.ones_like(inputs)
        updates = tf.zeros(tf.shape(indices_to_mask), dtype=inputs.dtype)
        mask = tf.tensor_scatter_nd_update(mask, full_indices, updates)  # Set n lowest values to 0

        # # Apply the mask to the tensor
        masked_tensor = inputs * mask

        return masked_tensor
        # return inputs

### Create Model

In [18]:
model = Sequential() #line by line from top to bottom vanilla, dense, fully connected
model.add(Dense(len(x[0]), activation='relu', input_shape=x[0].shape)) #fully connected, dense
model.add(CustomDropoutLayer(0.5)) #small values only --> small = not significant
model.add(Dense(6, activation='relu')) #hidden layer
model.add(CustomDropoutLayer(0.5)) #small values only --> small = not significant
model.add(Dense(len(y[0,:]), activation='sigmoid',)) #sigmoid, tanh, step, SGD, softmax

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #adam

In [19]:
log_custom_dropout=model.fit(x,y,epochs=2,batch_size=150) #backpropagation #too

Epoch 1/2
col 4
inputs [[2.82944393 4.09736681 0.731286466 0.407090187]
 [2.63386917 3.25245523 0 0]
 [2.19471335 2.64691353 0 0]
 ...
 [2.61263418 3.41927958 0.377749145 0.12983036]
 [1.92905378 1.55406499 0 0.352644682]
 [2.63801885 3.14638853 0 0]]
col type <class 'tensorflow.python.framework.ops.Tensor'>
num_to_drop_per_row 2
indices_to_mask [[3 2]
 [2 3]
 [2 3]
 ...
 [3 2]
 [2 3]
 [2 3]]
masked tensor [[2.82944393 4.09736681 0 0]
 [2.63386917 3.25245523 0 0]
 [2.19471335 2.64691353 0 0]
 ...
 [2.61263418 3.41927958 0 0]
 [1.92905378 1.55406499 0 0]
 [2.63801885 3.14638853 0 0]]
col 6
inputs [[0 4.37768221 0.854295433 0 1.99679637 2.34033442]
 [0 3.65279627 0.758677959 0 1.79189217 1.86621416]
 [0 2.99620056 0.628063 0 1.48559165 1.51988196]
 ...
 [0 3.76848269 0.76512897 0 1.80043209 1.95851946]
 [0 2.05277872 0.501750886 0.266696632 1.21378922 0.906354964]
 [0 3.57495546 0.752634525 0 1.78147531 1.80732238]]
col type <class 'tensorflow.python.framework.ops.Tensor'>
num_to_drop_pe

In [17]:
log_custom_dropout=model.fit(x,y,epochs=900,batch_size=150) #backpropagation #too

Epoch 1/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3333 - loss: 3.2536
Epoch 2/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.3333 - loss: 3.2084
Epoch 3/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.3333 - loss: 3.1736
Epoch 4/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.3333 - loss: 3.1260
Epoch 5/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.3333 - loss: 3.0918
Epoch 6/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.3333 - loss: 3.0579
Epoch 7/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.3333 - loss: 3.0125
Epoch 8/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.3333 - loss: 2.9792
Epoch 9/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 0.3333 - loss: 2.9341
Epoch 10/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.3333 - loss: 2.9014
Epoch 11/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - accuracy: 0.3333 - loss: 2.8691
Epoch 12/900
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.